In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import math

import time

from datetime import date # Removed duplicate import of datetime
import calendar

import nltk
nltk.download() # Commented out, as it may prompt a download dialog

import string
import regex # Removed duplicate import of re
import re

from nltk.corpus import stopwords
from pprint import pprint

import spacy

import contractions

# Text to emotion 
import nltk
nltk.download('punkt') # Moved download to correct location

import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests

from LeXmo import LeXmo

import tqdm
from tqdm import tqdm_notebook,tqdm

from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

import operator

# Increase display options for Pandas dataframes
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000
pd.options.display.max_colwidth = 10000
pd.options.display.max_seq_items = 2000


In [12]:
# Read databases
df_news_true=pd.read_csv('data/DataSet_Misinfo_TRUE.csv',index_col=False)
df_news_fake=pd.read_csv('data/DataSet_Misinfo_FAKE.csv')
df_news_rpsub=pd.read_csv('data/EXTRA_RussianPropagandaSubset.csv')

In [23]:
# Separate each paragraph into sentences using nltk.sent_tokenize
df_sentences=pd.DataFrame(columns=['sent'])
list_token=[]
for para in tqdm(df_news_fake.text):
    if pd.isna(para)==False:
        tokens = nltk.sent_tokenize(para)
        list_token=list_token+tokens

# print the number of sentences        
print("we have {} sentences in the Text".format(len(list_token)))

# Print the number of unique sentences       
print("we have {} unique sentences in the Text".format(len(set(list_token))))

#convert the list of sentences into dataframe 
df_sentence =pd.DataFrame([set(list_token)]).T

# Rename the column 
df_sentence.columns=['sent']

# Print the first sentence
print ( "The first sentence is:")
print(df_sentence.loc[1,'sent'] )

100%|██████████| 43642/43642 [04:52<00:00, 149.41it/s]


we have 749699 sentences in the Text
we have 526615 unique sentences in the Text
The first sentence is:
The Christian Post picked up the story and video interview, in which Assaf narrates the final days of her 18-year-old’s life.


In [32]:
# Add columns for each emotion to the df_sentence DataFrame
df_sentence['anger']=0
df_sentence['anticipation']=0
df_sentence['disgust']=0
df_sentence['fear']=0
df_sentence['joy']=0
df_sentence['negative']=0
df_sentence['positive']=0
df_sentence['sadness']=0
df_sentence['surprise']=0
df_sentence['trust']=0

# Fix contractions like "I'm" to "I am"
df_sentence['sent_cont_fixed']=''
for i,row in df_sentence.iterrows():
    # Use a try-except block to handle errors for non-English sentences
    try:
        df_sentence.loc[i,'sent_cont_fixed']=contractions.fix(row.sent)
    
    # If an error is caught, print the index and row for reference
    except:
        print(i,row)
    
# keep a copy of df_sentence
df_sentence_new=df_sentence.copy()

155516 sent               ‘ARTIK İŞ KAPIYA GELDİKTEN SONRA MÜDAHALE DÖNEMİ BİTTİ' \nErdoğan, ‘terörle mücadele' konusuna değinerek savunmadan taarruza geçtiklerini belirterek "Artık tehditlerin kapımıza dayanmasını beklemeyeceğimiz.
anger                                                                                                                                                                                                                           0
anticipation                                                                                                                                                                                                                    0
disgust                                                                                                                                                                                                                         0
fear                                                                                     

In [ ]:
# for each sentence, calculate the the related emotions and fill the values 
for i,row in tqdm_notebook(df_sentence_new.iterrows()):
     # Use a try-except block to handle errors for null sentences
    try:
        emo=(LeXmo.LeXmo(row.sent_cont_fixed))
        emo.pop('text', None)
        for key,value in emo.items():
            df_sentence_new.loc[i,key]=round(value,3)  
    except:
        print(i,row)

In [ ]:
#save the database with the emotion for each sentence
df_sentence_new.to_hdf('data/df_news_fake_emotions.h5', key='df_news_fake_emotions')

In [ ]:
df_sentence_new

In [ ]:
print ('l')